In [1]:
pip install langchain


[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain_experimental


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 3.0 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
##Load API key ------------------
f = open('../apikeys/api.txt', 'r')
key = f.readline()
os.environ['OPENAI_API_KEY'] = key

In [15]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.1, max_tokens=512, model="text-davinci-003")

from langchain.llms import OpenAI
llm = OpenAI(temperature=1, max_tokens=512, model="text-davinci-003")

In [16]:

sudoku_puzzle =   "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"
problem_description = f"""
{sudoku_puzzle}

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.
""".strip()
print(problem_description)

3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.


In [17]:
#######
# The following code implement a simple rule based checker for 
# a specific 4x4 sudoku puzzle.
#######

from typing import Tuple
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity
import re

In [18]:
class MyChecker(ToTChecker):
    def evaluate(self,
        problem_description: str,
        thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        
        last_thought = thoughts[-1]
        clean_solution = last_thought.replace(" ", "").replace('"', "")
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        else:
            return ThoughtValidity.INVALID


In [19]:
#######
# Testing the MyChecker class above:
#######

checker = MyChecker()
assert checker.evaluate("", ("3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1",)) == ThoughtValidity.VALID_FINAL
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1",)) == ThoughtValidity.INVALID


In [21]:
#######
# Initialize and run the ToT chain, 
# with maximum number of interactions k set to 30 and 
# the maximum number child thoughts c set to 8.
#######

from langchain_experimental.tot.base import ToTChain

tot_chain = ToTChain(llm=llm, checker=MyChecker(), k=50, c=5, verbose=True, verbose_llm=False)
tot_chain.run(problem_description=problem_description)



> Entering new ToTChain chain...
Starting the ToT solve procedure.
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,*,*
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,1,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,*
        Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,1
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,4
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,1,4
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,1,4
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,2,4
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,3,4
        Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,4
        Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,1,2
        Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,1,4
        Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,4,2
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,3,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,4,2,*
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1
    Thought: 3,1,*,2|1,*,3,*|*,1,*,3|4,*,*,1
    Thought: 3,*,2,2|1,*,3,*

'3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1'